In [ ]:
%pip install neo4j yfinance python-dotenv

In [ ]:
import os
import yfinance as yf
from neo4j import GraphDatabase
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# 1. Connection Config
URI = "bolt://localhost:7687"
AUTH = ("neo4j", os.getenv("NEO4J_PASSWORD"))
driver = GraphDatabase.driver(URI, auth=AUTH)

def ingest_financial_data(tickers):
    with driver.session() as session:
        for symbol in tickers:
            # Fetch real data from Yahoo Finance
            stock = yf.Ticker(symbol)
            info = stock.info
            
            # Extract basic entities
            name = info.get('longName', symbol)
            sector = info.get('sector', 'Unknown')
            summary = info.get('longBusinessSummary', 'No summary available.')

            # Create Nodes and Relationships in Neo4j
            session.run("""
            MERGE (c:Company {ticker: $ticker})
            SET c.name = $name, c.summary = $summary
            MERGE (s:Sector {name: $sector})
            MERGE (c)-[:IN_SECTOR]->(s)
            """, ticker=symbol, name=name, sector=sector, summary=summary)
            
            print(f"Ingested: {name} in {sector} sector.")

def graph_rag_traversal(target_sector):
    """
    Example of 'Graph Retrieval Fusion':
    Find all companies in a sector and retrieve their summaries for RAG context.
    """
    query = """
    MATCH (s:Sector {name: $sector})<-[:IN_SECTOR]-(c:Company)
    RETURN c.name AS company, c.summary AS context
    """
    with driver.session() as session:
        results = session.run(query, sector=target_sector)
        return [f"Data for {r['company']}: {r['context'][:200]}..." for r in results]


In [ ]:
# Example usage
print("--- Starting Ingestion ---")
tickers = ["AAPL", "MSFT", "TSLA", "XOM"]
ingest_financial_data(tickers)

print("\n--- Graph RAG Retrieval ---")
tech_context = graph_rag_traversal("Technology")
for item in tech_context:
    print(item)